In [1]:
from gcpspark import *
import pyspark.sql.functions as F
from pyspark import StorageLevel
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
import os

Dotenv installed
Loaded variables .env True
Pyspark installed
JAVA:  /usr/lib/jvm/temurin-8-jdk-amd64
DATAPROC:  True
Current GCP Project Name: cencosudx
Current ENVIRONMENT: staging
<class 'Exception'>


In [2]:
spark = create_pyspark(name="co-contribucion4" , 
                       connection="gcp")

File already exists at: /jars/spark-bigquery-with-dependencies_2.12-0.26.0.jar
File already exists at: /jars/gcs-connector-hadoop3-2.2.19.jar
Process: co-contribucion4_1746117651496


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/01 16:40:55 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
25/05/01 16:40:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
25/05/01 16:40:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
25/05/01 16:40:55 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
df = spark.read.parquet("gs://staging-da-contribution/CO/mongo_alerts")
df = df.filter(F.col("updated_at")>=datetime(2024,1,1))
df = df.filter(F.col("state")=="COMPLETED")

In [4]:
# display(df
#         .filter(F.col("type")=="premerma")
#         .filter(F.month("updated_at")=="01")
#         .filter(F.dayofmonth("updated_at")=="14")
#        )

In [5]:
df = df.withColumn("updated_hour",F.hour("updated_at"))\
        .withColumn("updated_at",F.date_format("updated_at","yyy-MM-dd"))

In [6]:
#display(df)

In [7]:
df_final = df.select(F.col("ean").alias("EAN"),
                     F.col("sku").alias("Item_Id"),
                     F.col("updated_hour").alias("Hour_Alert"),
                     F.col("type").alias("Type"),
                     F.col("store").alias("Location_Id"),
                     F.col("updated_at").alias("Date").cast("string"),
                     F.col("sector").alias("Sector")
                    ).distinct()

In [8]:
#display(df_final)


In [9]:
df_final.printSchema()

root
 |-- EAN: string (nullable = true)
 |-- Item_Id: string (nullable = true)
 |-- Hour_Alert: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Location_Id: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Sector: string (nullable = true)



In [10]:
df_final.write.mode("overwrite").parquet("gs://staging-da-contribution/CO/final_alerts")

In [11]:
#df_final = spark.read.parquet("gs://staging-da-contribution/CO/final_alerts")

In [12]:
#display(df_final.groupBy("Type","Location_Id","EAN","Item_Id",F.month("Date"),F.year("Date")).count())

In [13]:
#display(df_final.groupBy("Type",F.month("Date"),F.year("Date")).count().filter(F.col("Type")=="premerma"))

In [14]:
spark.stop()

In [15]:
print("Done")

Done
